In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [1]:
import pandas as pd

In [7]:
df = pd.read_csv('/content/train.csv')

In [4]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM,Dense,Embedding,Bidirectional
from tensorflow.keras.models import Sequential

In [6]:
import nltk
from nltk.corpus import stopwords
import re

In [8]:
df.dropna(inplace=True)

In [9]:
df.reset_index(inplace=True)
X = df.drop('label',axis = 1)
Y = df['label']

In [13]:
X.shape

(18285, 3)

In [12]:
X.drop(['index','id'],axis = 1,inplace=True)

In [14]:
from nltk.stem.porter import PorterStemmer

In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [19]:
ps = PorterStemmer()
corpus= []
for i in range(0,len(X)):
  title = re.sub('^[a-zA-Z]',' ',X['title'][i])
  title = title.lower()
  title = title.split()
  title = [ps.stem(word) for word in title if not word in stopwords.words('english')]
  title = ' '.join(title)
  corpus.append(title)

In [21]:
voc_size = 5000
one_hot_repr = [one_hot(words,voc_size) for words in corpus]

In [23]:
max_size = 20
embedded_repr = pad_sequences(one_hot_repr,padding='pre',maxlen=max_size)

In [24]:
embedded_repr

array([[   0,    0,    0, ..., 2210, 2794, 4474],
       [   0,    0,    0, ..., 3495, 3067, 1144],
       [   0,    0,    0, ..., 4395, 4389, 2277],
       ...,
       [   0,    0,    0, ...,  626, 3607, 4113],
       [   0,    0,    0, ..., 3754,  252, 2609],
       [   0,    0,    0, ..., 1305, 2453, 3917]], dtype=int32)

In [41]:
embedding_features = 40
model = Sequential()
model.add(Embedding(voc_size,embedding_features,input_length=max_size))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [42]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dense_2 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________


In [30]:
import numpy as np
X_final = np.array(embedded_repr)

In [43]:
model.fit(X_final,Y,validation_split=0.2,epochs=10,batch_size=72)

Epoch 1/10
204/204 [==============================] - 6s 14ms/step - loss: 0.2877 - accuracy: 0.8609 - val_loss: 0.2096 - val_accuracy: 0.9169
Epoch 2/10
204/204 [==============================] - 2s 10ms/step - loss: 0.1315 - accuracy: 0.9495 - val_loss: 0.2197 - val_accuracy: 0.9147
Epoch 3/10
204/204 [==============================] - 2s 10ms/step - loss: 0.0875 - accuracy: 0.9681 - val_loss: 0.2435 - val_accuracy: 0.9109
Epoch 4/10
204/204 [==============================] - 2s 10ms/step - loss: 0.0597 - accuracy: 0.9783 - val_loss: 0.3529 - val_accuracy: 0.9109
Epoch 5/10
204/204 [==============================] - 2s 10ms/step - loss: 0.0366 - accuracy: 0.9878 - val_loss: 0.3586 - val_accuracy: 0.9057
Epoch 6/10
204/204 [==============================] - 2s 10ms/step - loss: 0.0235 - accuracy: 0.9921 - val_loss: 0.4197 - val_accuracy: 0.9109
Epoch 7/10
204/204 [==============================] - 2s 10ms/step - loss: 0.0134 - accuracy: 0.9957 - val_loss: 0.5374 - val_accuracy: 0.9068

In [44]:
from sklearn.model_selection import train_test_split 
x_train,x_test,y_train,y_test  =train_test_split(X_final,Y,test_size = 0.25)

In [45]:
x_test,y_test

(array([[   0,    0,    0, ..., 3995,  251, 1602],
        [   0,    0,    0, ..., 2524, 1691, 3932],
        [   0,    0,    0, ..., 2812, 4976, 1144],
        ...,
        [   0,    0,    0, ...,  626, 3607, 4113],
        [   0,    0,    0, ..., 2975, 1188, 3960],
        [   0,    0,    0, ..., 1516, 1110, 1258]], dtype=int32), 12748    1
 13114    1
 10201    0
 13762    0
 16       1
         ..
 15055    0
 3819     1
 16098    0
 10125    1
 882      1
 Name: label, Length: 4572, dtype: int64)

In [46]:
pred = model.predict(x_test)

In [47]:
model.evaluate(x_test, y_test, verbose=2)

143/143 - 0s - loss: 0.1115 - accuracy: 0.9768 - 461ms/epoch - 3ms/step


[0.1114639937877655, 0.976815402507782]